<a href="https://colab.research.google.com/github/AlphaLock/Career_Accelerator_P5_NLP_Project_Work/blob/main/Azubi_Sentiment_Analysis_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

In [ ]:
import pandas as pd
import numpy as np

#from datasets import load_metric
import os
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
import io
from transformers import AutoTokenizer, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
from google.colab import files
from google.colab import drive
from transformers import AutoModelForSequenceClassification

#converting training data to PyTorch tensors to speed up training and adding padding:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
from huggingface_hub import notebook_login

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
drive.mount('/content/drive')

## Data set importation

In [ ]:
train =pd.read_csv("/content/drive/MyDrive/Azubi/hugging.csv").dropna(axis = 0)
test = pd.read_csv("/content/drive/MyDrive/Azubi/Testhugging.csv").fillna("")


## Data cleaning

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()

In [ ]:
#Removing missing values
train.dropna(inplace = True)

In [ ]:
train.isnull().sum()

In [ ]:
# Remove customer IDs from the data set
train = train.iloc[:,1:]

In [ ]:
train_data.head()

Class Distribution Analysis

Examine the distribution of the "label" column to understand the balance between different classes (e.g., positive, negative, neutral)

In [ ]:
ax = (train_data['label'].value_counts()*100.0 /len(train_data))\
.plot.pie(autopct='%.1f%%', labels = ['Neutral', 'Positive', 'Negative'],figsize =(5,5), fontsize = 12 )
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel('',fontsize = 12)
ax.set_title('Distribution of tweet', fontsize = 12)

plt.show()

Agreement Analysis:

Investigate the "agreement" column to understand the level of agreement between annotators or classifiers. Analyze the distribution of agreement scores and determine if there are discrepancies or inconsistencies.

In [ ]:
ax = train_data['agreement'].value_counts().plot(kind = 'bar',rot = 45, width = 0.3)
ax.set_ylabel('Agreement')
ax.set_title('Agreement Distribution')

plt.show()

Text Length Analysis

Explore the length of the "safe_text" column (number of characters or words).

In [ ]:
# Calculate the length of each text
train_data['text_length'] = train_data['safe_text'].apply(lambda x: len(x))

# Plot the text length distribution
plt.hist(train_data['text_length'], bins=20)
plt.xlabel('Text Length')
plt.ylabel('Count')
plt.title('Text Length Distribution')
plt.show()

In [ ]:
# modeling

In [ ]:
train, eval = train_test_split(train, test_size=0.2, random_state=42, stratify= train['label'])

In [ ]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

In [ ]:
# Save splitted subsets
train.to_csv("/content/drive/MyDrive/Azubi/train_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/Azubi/eval_subset.csv", index=False)

In [ ]:
dataset = load_dataset('csv',
                        data_files={'train': '/content/drive/MyDrive/Azubi/train_subset.csv',
                        'eval': '/content/drive/MyDrive/Azubi/eval_subset.csv'}, encoding = "ISO-8859-1")

In [ ]:
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}


In [ ]:
def tokenize_data(example):
    return tokenizer(example['safe_text'], padding='max_length')

In [ ]:
# Change the tweets to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)


In [ ]:
training_args = TrainingArguments("test_trainer", num_train_epochs= 10, load_best_model_at_end=True,evaluation_strategy= "steps")

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

train_dataset = dataset['train'].shuffle(seed=20) #.select(range(40000)) # to select a part
eval_dataset = dataset['eval'].shuffle(seed=20)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

trainer.train()

In [ ]:
#defining the evaluation metrics
metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset,
                  eval_dataset=eval_dataset,compute_metrics=compute_metrics)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Launch the final evaluation
trainer.evaluate()